<a href="https://colab.research.google.com/github/Kamurasi-Jordan-Arthur/Year-II-AI/blob/main/21U08812EVE_ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 21/U/08812/EVE    AND     21/U/12682
#### Kamurasi jordan arthur                 //             Alabyeekobo Suubi Brain

## Objective 
- Objective is to try a predict the co2emmisions given feature composition
- Also used additional clustering algorithm to find the the hidden relation ship between the highwaympg and year over the years
### Below are the description of the data set 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
veichles = pd.read_csv("/content/vehicles.csv")
# bikes = pd.read_csv("bikes.csv")

FileNotFoundError: ignored

## A veiw of my chossen data set

In [ ]:
veichles.head(5)
# bikes.head()

# Data Exploration

In [ ]:
veichles.info()
veichles.describe()

In [ ]:
veichles.isna().sum()

In [ ]:
for x in veichles.columns:
    if veichles[x].dtype == 'object':
        print(veichles[x].describe())
        print('\n\n')



## Data set Visualisation
### mean C02 emissions as per number of cylinders

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
minmax = MinMaxScaler()
x = veichles.groupby('cylinders')['co2emissions'].mean().reset_index()

# x['co2emissions']=minmax.fit_transform(x['co2emissions'].to_frame())
x.plot(kind = 'bar', x = 'cylinders', y = 'co2emissions', color = 'black')
x.describe()

## Preprocessing

In [ ]:
from scipy import stats

le = LabelEncoder()
x = veichles.pivot(columns = 'drive', values = 'co2emissions')
x.plot(kind = 'box', figsize = (10,6))

In [ ]:
plt.scatter(veichles['citympg'],veichles['highwaympg'])
plt.show()

In [ ]:
veichles.isnull().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler
le = LabelEncoder()

# droping columns that dont contribute tthat much to the model according to the above correlation
# veichles.drop({'make','model','transmissionspeeds','year'}, axis = 1, inplace = True)

veichles.loc[:,('year','cylinders','transmissionspeeds','drive','class','transmissiontype')] = (veichles.loc[:,('year','cylinders','transmissionspeeds','drive','class','transmissiontype')].astype('category'))
# veichles.drop(['year','cylinders'], axis = 1)

cat_columns = veichles.loc[:,('year','cylinders','transmissionspeeds','drive','class','transmissiontype')].columns
for x in cat_columns:
    veichles[x] = le.fit_transform(veichles[x])

stdScl = StandardScaler()
veichles.loc[:,(x for x in veichles.columns if veichles[x].dtype != 'object')] = stdScl.fit_transform(veichles.loc[:,(x for x in veichles.columns if veichles[x].dtype != 'object')])


### Veiw of the correlation to find best feature to use when filling the NA vaues

In [ ]:
veichles.corr()


### Dealing with null values

In [ ]:
mask = veichles['citympg'].isnull() & veichles['highwaympg'].isnull()
mask0 = veichles['citympg'].isnull() & veichles['highwaympg'].isnull()

print('Rows with \'NA\' in both \n citympy and highwaympg are:- {}. \n cylinders and displacement are:- {}'.format(mask.sum(), mask0.sum()))

# function to derive values for NA values in the data from features with high correlation
def fillna_value(y):
    x,y = None,None
    
    
    if y == 'citympg':        
        x = veichles['highwaympg'].dropna().to_numpy()
        y = veichles['citympg'].dropna().to_numpy()
        x.resize(y.shape)
        y.resize(y.shape)
                
    elif y == 'highwaympg':      
        x = veichles['citympg'].dropna().to_numpy()
        y = veichles['highwaympg'].dropna().to_numpy()
        x.resize(x.shape)
        y.resize(x.shape)
    else:        
        x = veichles['cylinders'].dropna().to_numpy()
        y = veichles['displacement'].dropna().to_numpy()
        x.resize(y.shape)
        y.resize(y.shape)
    a,b ,c ,d ,e = stats.linregress(x, y)   
    return lambda x : x*a + b


veichles.fillna({'citympg': fillna_value('citympg')(veichles['highwaympg']),
                 'highwaympg': fillna_value('highwaympg')(veichles['citympg']),
                 'displacement': fillna_value('displacement')(veichles['cylinders']),
                }, inplace = True)

veichles.head()

### Performing feature reduction with feature the model wont rely on that much

In [ ]:
veichles.drop(['model','make','class'], axis = 1 ,inplace = True)
le2 = LabelEncoder()
veichles['co2_range'] = le2.fit_transform(pd.cut(veichles['co2emissions'], bins = [-6.498927e-01,6.697254e-01,6.732510e+0]))
# sns.pairplot(veichles, hue = 'co2_range')

In [ ]:
veichles.groupby('co2_range')['co2emissions'].mean().plot(kind = 'bar')

# Working On Models

### Spliting the data

In [ ]:
from sklearn.model_selection import train_test_split

X = veichles.iloc[:,:8]
Y = veichles.iloc[:,8:]

x_train,x_test,y_train, y_test = train_test_split(X,Y, random_state = 29, test_size = .25)


## logistic Regressor 

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
LoR = LogisticRegression(random_state = 1)
LoR.fit(x_train,y_train['co2_range'])

In [ ]:
y_pred = LoR.predict(x_test)
print('Model accuracy_score : ', accuracy_score(y_test['co2_range'],y_pred)*100)
print('Model mean_squared_error : ', mean_squared_error(y_test['co2_range'],y_pred)*100)
print('Model mean_absolute_error: ', mean_absolute_error(y_test['co2_range'],y_pred)*100)


In [ ]:
classification_report(y_test['co2_range'],y_pred)

## Linear Regressor

In [ ]:
LiR = LinearRegression()
LiR.fit(x_train.drop(['transmissiontype','year'], axis = 1),y_train['co2emissions'])
# rom sklearn.linear_model import LinearRegression
# reg = LinearRegression()
# parameters = {"alpha": [1, 10, 100, 290, 500], "fit_intercept": [True, False], "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], }
# grid = GridSearchCV(estimator=reg, param_grid = parameters, cv = 2, n_jobs=-1)
# grid.fit(x_train, y_train)
# reg.score(x_test ...

In [ ]:
y_pred = LiR.predict(x_test.drop(['transmissiontype','year'], axis = 1))

print('model Mean_absolute_error : ', mean_absolute_error(y_test['co2emissions'] , y_pred))
print('model Mean_squared_error : ', mean_squared_error(y_test['co2emissions'] , y_pred))
print('model R2_score : ', LiR.score(x_test.drop(['transmissiontype','year'], axis = 1), y_test['co2emissions'].to_numpy()))

# plt.scatter(y_test['co2emissions'],y_pred)
# plt.title('Y-predict Vs y-actual')
# plt.xlabel('co2emissions')
# plt.ylabel('predicted emmisions')

# N.B
### linear  and logistic models are provded as aready well turned models 
##### Furter turning of features only reduces model  performence 

## Decision Tree Usage

#### Turning the Criterion Feature

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
criterion = ['gini', 'entropy', 'log_loss']
criterion1 = []
clf = DecisionTreeClassifier()

for x in criterion:
    clf = DecisionTreeClassifier( criterion = x )
    clf.fit(x_train, y_train['co2_range'])
    y_pred = clf.predict(x_test)
    criterion1.append(accuracy_score(y_pred,y_test['co2_range']))
criterion1

### Fine Turning the Max_Depth paramerter

In [ ]:
clr = DecisionTreeRegressor()
score = []
depths = pd.DataFrame(range(7,52),columns = ['Max_depth'])

for x in depths.values:
    clr = DecisionTreeRegressor(max_depth = int(x))
    clr.fit(x_train, y_train['co2emissions'])
    y_pred = clf.predict(x_test)
    score.append(clr.score(x_test, y_test['co2emissions']))
    
depths['Model_score'] = np.asarray(score).reshape(-1,1)


In [ ]:
depths.plot(kind = 'line' ,x = 'Max_depth', y = 'Model_score',title = "Line graph showing model perfomences with depth")

## Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# rfr = RandomForestRegressor(criterion  ='entropy', max_leaf )
rfr = RandomForestRegressor()
rfr.fit(x_train,y_train['co2emissions'])
rfr.get_params()

In [ ]:
from sklearn.metrics import r2_score
y_pred = rfr.predict(x_test)

### Random Forest model metrix using defualt parameters

In [ ]:
print('Data_stdeviation and Mean;', y_test['co2emissions'].std(), y_test['co2emissions'].mean() )
print('Predict_stdeviation and Mean ',y_pred.std() , y_pred.mean())
print('model\'s R2_score: ', r2_score(y_test['co2emissions'],y_pred))


### Using GridSearchCV Search basing on to  fine tune parameters
                                                                                            21/U/08812/EVE

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [80, 170, 300],
    'max_depth': [2, 3 , 7],
    'min_samples_split': [3, 4, 7],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt', 'log2']
}

gs_cv = GridSearchCV(estimator=rfr, param_grid=param_grid,
                     cv=3, n_jobs=3)


In [ ]:
gs_cv.fit(x_train, y_train['co2emissions'])


In [ ]:
print('best_params:',gs_cv.best_params_)

In [ ]:
rfr.set_params(**gs_cv.best_params_)
rfr.fit(x_train,y_train['co2emissions'])
new_y_pred = rfr.predict(x_test)

print('Previous_std_value and Mean;', y_pred.std() , y_pred.mean() )
print('New_std_value and Mean ',new_y_pred.std() , new_y_pred.mean())
print('New_model\'s R2_score: ', r2_score(y_test['co2emissions'],new_y_pred))



## K-means clustering

## Parameter hyper turning
                                                                                            21/U/08812/EVE

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans()
param_grid = {
    'n_clusters': [2, 3, 4, 5],
    'init': ['k-means++', 'random'],
    'n_init': [10, 20, 30],
    'max_iter': [130, 300,420]
         }
gs_cv0 = GridSearchCV(estimator=kmeans, param_grid=param_grid,
                     cv=3, n_jobs=4)
X = x_train.loc[:, ['year','highwaympg']]
gs_cv0.fit(X)
best_params = gs_cv.best_params_
print('the best parameters:', best_params)

In [ ]:
kmeans.set_params(**gs_cv0.best_params_)
kmeans.fit(X)

In [ ]:
X['pred'] = kmeans.predict(x_test.loc[:, ['year','highwaympg']])

X.plot(kind = 'scatter', x = 'highway',y ='year', hue='pred')


# Best performing model
### Random forest classifier

In [ ]:
print('Model_std_value and Mean ',new_y_pred.std() , new_y_pred.mean())
print('Model_model\'s R2_score: ', r2_score(y_test['co2emissions'],new_y_pred))
print('MSE', mean_squared_error(y_test['co2emissions'],new_y_pred))
print('RMSE', np.sqrt(mean_squared_error(y_test['co2emissions'],new_y_pred)))


